In [91]:
import numpy as np

In [92]:
#   .txt
file_name = 'test.txt'

### Deliverable 1


In [93]:
file = open(file_name, 'r')
content = file.read()
file.close()

num_char = len(content)

unique_char = set(content)
num_unique_char = len(unique_char)

# each character is encoded by the same number of bits
average_bit = np.round((np.log(num_unique_char)), 2)
total_bit = average_bit * num_char

# print(content, end='\n\n')
# print(unique_char, end='\n\n')

print('Total number of characters: {} \nNumber of distinct characters: {} \nTotal number of bits needed to encode the text: {} \nAverage number of bits per character: {}'.format(
    num_char, num_unique_char, total_bit, average_bit))

Total number of characters: 1799 
Number of distinct characters: 42 
Total number of bits needed to encode the text: 6728.26 
Average number of bits per character: 3.74


### Deliverable 2


In [94]:
file = open(file_name, 'r')
content = file.read()
file.close()

num_char = len(content)

char_freq = dict()
for char in content:
    char_freq[char] = char_freq.get(char, 0) + 1

# sort unique characters by frequency in descending order
sorted_unique_chars = sorted(
    char_freq.keys(), key=lambda char: char_freq[char], reverse=True)

print(sorted_unique_chars)

total_bit = 0
i = 1
for char in sorted_unique_chars:
    total_bit += char_freq[char] * i
    i += 1

average_bit = np.round(total_bit / num_char, 2)

print('Total numer of bits: {} \nAverage number of bits: {}'.format(
    total_bit, average_bit))

[' ', 'e', 't', 'o', 'a', 'i', 's', 'h', 'n', 'r', 'd', 'l', 'f', 'g', 'u', 'c', 'w', 'm', 'b', 'p', '\n', 'y', '.', ',', 'v', 'k', '’', 'T', 'H', '—', '?', 'S', 'O', 'C', '-', 'F', 'A', 'B', 'I', 'q', 'x', 'N']
Total numer of bits: 14754 
Average number of bits: 8.2


### Deliverable 3


In [95]:
class Node:
    def __init__(self, chars, freq, left=None, right=None):
        self.chars = chars
        self.freq = freq
        self.left = left
        self.right = right

In [96]:
def get_char_freq(file_name):
    file = open(file_name, 'r')
    content = file.read()
    file.close()

    char_freq = dict()
    for char in content:
        char_freq[char] = char_freq.get(char, 0) + 1

    return char_freq

In [97]:
def create_node(char_freq):
    nodes = list()

    for char, freq in char_freq.items():
        nodes.append(Node([char], freq))

    return nodes

In [98]:
def merge_trees(left_root, right_root):
    chars = list(set(left_root.chars + right_root.chars))
    freq = left_root.freq + right_root.freq

    return Node(chars, freq, left_root, right_root)

In [99]:
def create_binary_tree(nodes):
    sorted_nodes = sorted(nodes, key=lambda x: x.freq)

    root = sorted_nodes[0]
    for i in range(1, len(sorted_nodes), 1):
        root = merge_trees(sorted_nodes[i], root)

    return root

In [100]:
# HELPER
def visualise_tree(root, level=0):
    print(('\t' * level) + '{} - {} [{}]'.format(level, root.chars, root.freq))

    level += 1
    if root.left:
        visualise_tree(root.left, level)

    if root.right:
        visualise_tree(root.right, level)

In [101]:
test_case = {'a': 20, 'b': 2, 'c': 1}
root_test_case = create_binary_tree(create_node(test_case))

visualise_tree(root_test_case)

0 - ['a', 'b', 'c'] [23]
	1 - ['a'] [20]
	1 - ['b', 'c'] [3]
		2 - ['b'] [2]
		2 - ['c'] [1]


### Deliverable 4


In [103]:
def find_in_tree(char, root):
    level = 0
    while root:
        if not root.left:
            if char == root.chars[0]:
                return level
            else:
                break

        level += 1
        if char == root.left.chars[0]:
            return level

        root = root.right

    return 'None'

In [104]:
find_in_tree('a', root_test_case)

1

In [105]:
find_in_tree('b', root_test_case)

2

In [106]:
find_in_tree('c', root_test_case)

2

In [107]:
find_in_tree('d', root_test_case)

'None'

### Deliverable 5


In [108]:
def tree_from_text(file_name):
    file = open(file_name, 'r')
    content = file.read()
    file.close()

    char_freq = dict()
    for char in content:
        char_freq[char] = char_freq.get(char, 0) + 1

    unique_char_sorted = sorted(
        char_freq.keys(), key=lambda char: char_freq[char], reverse=True)

    total_bit = 0
    num_char = len(content)
    root = Node(unique_char_sorted, num_char)
    for i in range(0, len(unique_char_sorted), 1):
        char = unique_char_sorted[i]

        total_bit += char_freq[char] * (i + 1)

        root.left = Node([char], char_freq[char])
        root.right = Node(unique_char_sorted[i+1:], num_char - char_freq[char])

        root = root.right
        if len(root.chars) == 1:
            break

    average_bit = np.round(total_bit / num_char, 2)

    print('Total number of bits: {} \nAverage number of bits: {}'.format(
        total_bit, average_bit))

    return root

In [109]:
tree_from_text(file_name)

Total number of bits: 14712 
Average number of bits: 8.18
